## Univariate Timeseries Classification

In [1]:
import os
import pandas as pd

filename = os.path.join("data", "arrowhead", "ArrowHead.txt")
data = pd.read_csv(filename, sep='  ', header=None, engine="python")

X_train_orig = data.iloc[:, 1:]
y_train_orig = pd.get_dummies(data.iloc[:, 0].astype('int'))

In [2]:
from pyreal.transformers import DimensionAdder

formatter = DimensionAdder()

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.get_logger().setLevel('INFO')

model = keras.Sequential()
model.add(layers.LSTM(128, input_shape=(X_train_orig.shape[1], 1)))
model.add(layers.BatchNormalization())
model.add(layers.Dense(3))

model.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer="adam",
    metrics=["accuracy"],
)


In [ ]:
model.fit(formatter.transform(X_train_orig), y_train_orig, epochs=5, batch_size=16)

Epoch 1/5


In [ ]:
import importlib
import pyreal.explainers.time_series.saliency.univariate_occlusion_saliency as oi
import time

transformers = [formatter]

explainer = oi.UnivariateOcclusionSaliency(model, X_train_orig, transformers=transformers, width=5, k=0, classes=[1, 2, 3])
explainer.fit()

start = time.time()
explanation1 = explainer.produce(X_train_orig.iloc[0])
explanation2 = explainer.produce(X_train_orig.iloc[1])

In [ ]:
import pyreal.utils.visualize as vis

vis.plot_timeseries_saliency(X_train_orig.iloc[0], explanation1[0].iloc[0, :])